<div style="text-align: center; margin: 50px">

<h1 style="text-align: center;">Qubit by Qubit</h1>
<h2 style="text-align: center;">Introduction to Quantum Computing</h1>
<h3>Week 22 - Tunable Quantum Circuits and Solving the Knapsack Problem</h3>

</div>

Let's make the needed imports:

In [1]:
pip install qiskit

     |████████████████████████████████| 6.5 MB 2.6 MB/s 
     |████████████████████████████████| 18.0 MB 93 kB/s 
     |████████████████████████████████| 238 kB 47.0 MB/s 
     |████████████████████████████████| 200 kB 43.8 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 1.6 MB 49.1 MB/s 
     |████████████████████████████████| 943 kB 42.4 MB/s 
     |████████████████████████████████| 49 kB 5.2 MB/s 
     |████████████████████████████████| 37.5 MB 1.3 MB/s 
     |████████████████████████████████| 38.1 MB 1.4 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 
     |████████████████████████████████| 3.6 MB 32.9 MB/s 
     |████████████████████████████████| 113 kB 36.3 MB/s 
  Created wheel for qiskit: filename=qiskit-0.34.2-py3-none-any.whl size=11805 sha256=08a247a6d7ecb5d04f86cee9780b525912a5ec9caa3c5e990c9a1863e546de57
  Stored in directory: /root/.cache/pip/wheels/62/77/65/cda6eedfdd2a525bd3f479a4386930ae3088a1eb01f8c94

In [3]:
# BLOCK 1 - Importing libaries. A lot of new libararies here! 

#from qiskit_optimization.applications import Knapsack
from qiskit.algorithms import NumPyMinimumEigensolver
#from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.utils import QuantumInstance
from qiskit import Aer
#from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit.library import EfficientSU2
print("All libraries imported successfully!")

All libraries imported successfully!


# Part 1 - Tunable Circuits

In the first part of this notebook, we will explore how tunable circuits are created in Qiskit. Then, in the second part of the lab, we will use these tunable circuits to solve the Knapsack problem using the VQE algorithm!

We will use one of Qiskit's built-in functions to create tunable circuits - `EfficientSU2`. We already imported this function above.

## Using the `EfficientSU2` function

The syntax for this function is as follows:

`tunable_circuit = EfficientSU2(parameters)`

As an example, the code below creates a tunable circuit using the EfficientSU2 function.

In [4]:
# Block 2 - Creating a tunable circuit using EfficientSU2
tunable_circuit = EfficientSU2(num_qubits = 2, reps=1, entanglement='linear',insert_barriers=True)

Let's visualize this circuit! We will need to use two functions to see the circuit - `decompose` and `draw`.

In [7]:
# Block 3 - Visualizing the tunable circuit
tunable_circuit.decompose().draw()

┌──────────┐┌──────────┐ ░       ░ ┌──────────┐┌──────────┐
q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[2]) ├─░───■───░─┤ Ry(θ[4]) ├┤ Rz(θ[6]) ├
     ├──────────┤├──────────┤ ░ ┌─┴─┐ ░ ├──────────┤├──────────┤
q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[3]) ├─░─┤ X ├─░─┤ Ry(θ[5]) ├┤ Rz(θ[7]) ├
     └──────────┘└──────────┘ ░ └───┘ ░ └──────────┘└──────────┘

### Questions:

1. How many qubits does this circuit have?
2. Which gates are used in the rotation stage? 
3. Which gate is used in the entanglement stage?

### Important parameters in `EfficientSU2`:

When defining a tunable circuit, we need to provide the following parameters to the `EfficientSU2` function:
1. The number of qubits, specified by `num_qubits`
2. The number of times the rotation and entanglement blocks should be repeated, specified by `reps`.
3. The type of entanglement, specified by `entanglement`. The options for this parameter are `full`, `linear`, and `circular`.

Additionally, we also specified `insert_barriers = True` to place barriers between each of the stages of the circuit and make it visually easier to distinguish the stages.

### Activity 1 - Create your own tunable circuit!

Use the `EfficientSU2` function to create a tunable circuit with the following specifications:
1. `num_qubits = 4`
2. `reps = 2`
3. `entanglement = linear`

Draw the circuit.

In [ ]:
# Block 4 - Create a tunable circuit with the specs defined above and draw it.

### Activity 2 - Exploring entanglement

Try changing the `entanglement` parameter to `full` or `circular` and redraw the circuit.

In [ ]:
# Block 5 - Redefine the tunable circuit with the same parameters as Block 4, but with entanglement = `full` or `circular` and draw it.

How did the circuit change when you changed `entanglement`?

`full` entanglement entangles each qubit with every other qubit, which gives you more tunability in your circuit. However, this involves using more CX gates, increasing the chance of errors. When implementing near-term algorithms, engineers often have to consider if `linear` entanglement is good enough, or if they need more.

# Part 2 - Solving the Knapsack problem

In this part, we will use the tunable circuits we created above to solve the knapsack problem using VQE!

The knapsack problem is a problem in combinatorial optimization: Given a set of items, each with a weight and a value, determine the number of each item to include in a collection so that the total weight is less than or equal to a given limit and the total value is as large as possible. It derives its name from the problem faced by someone who is constrained by a fixed-size knapsack and must fill it with the most valuable items. The problem often arises in resource allocation where the decision makers have to choose from a set of non-divisible projects or tasks under a fixed budget or time constraint, respectively.

## Part 1 - Setting up the problem

In this part, we will define the knapsack problem. Qiskit has a built-in function called `Knapsack` that we will use to define the problem. 

We define a list of values, a list of weights, the maximum weight, and then put it all together into a variable called `problem` using the Knapsack function.

In [ ]:
# BLOCK 6 - Defining the knapsack problem. We define a list of values, a list of weights, the maximum weight, and then
# put it all together into a variable called `problem' using the Knapsack function.
values = [4, 2, 5, 10]   # list of the values of items
weights = [3, 1, 2, 6]   # list of the weights of items
max_weight = 10            # maximum weight capacity (knapsack capacity)

knapsack_problem = Knapsack(values = values, weights = weights, max_weight = max_weight) #putting it all together

Feel free to be creative here - enter your own combination of values, weights, and max_weight! For a first pass, we recommend keeping the number of individual values < 5 . You can try bigger problems later!

## Part 2 - Using VQE to solve the problem

Let's solve the problem using VQE! Don't worry about how VQE works and the details of what is going on in this code - just try running it and see what results it produces.

In Block 7 below, we will convert the knapsack problem to a form that a quantum computer can understand, and also find out the number of qubits required to solve the problem.

How many qubits does your problem need?

In [ ]:
# BLOCK 7 - Converting the knapsack problem to a quantum circuit, i.e., a colletion of quantum gates (or operators).
# Run this block to convert the knapsack problem to a quantum operator, and print the number of qubits used.
operator, offset = QuadraticProgramToQubo().convert(knapsack_problem.to_quadratic_program()).to_ising()
print("number of qubits =",operator.num_qubits)

Next, we will set up VQE. We will specify the tunable circuit using `EfficientSU2`. YOU WILL NEED TO FILL CODE HERE!!!

We will also specify the classical optimizer we want to use. Remember that hybrid algorithms have a tunable quantum circuit, and a classical computer that tells it which parameters to use. This optimizer is the classical part of the hybrid algorithm.  We will use a popular classical optimization algorithm known as SPSA. You do not need to fill code here - we have filled it out for you.

We will use a built-in Qiskit function named `VQE` to put together all this information to solve the problem using the VQE algorithm. Again, we have completed this step for you.

In [ ]:
# BLOCK 8 - Setting up VQE.
# We define our tunable circuit for VQE. Here, we will choose EfficientSU2. YOU WILL NEED TO FILL CODE FOR THIS STEP
# We define which classical optimizer we want to use - here we will use one called SPSA
# We tell our code to use VQE with the tunable circuit, the optimizer, and the quantum instance

# FILL CODE HERE TO DEFINE THE TUNABLE QUANTUM CIRCUIT. SPECIFY THE CORRECT NUMBER OF QUBITS, AND CHOOSE THE NUMBER OF REPS AND THE LEVEL OF ENTANGLEMENT.


optimizer = SPSA(maxiter=15) # Classical optimizer

method = VQE(ansatz = tunable_circuit, optimizer = optimizer, quantum_instance = Aer.get_backend('qasm_simulator')) # Using the VQE algorithm 

Let's visualize the circuit you made!

In [ ]:
# Block 9 - Visualize your circuit by drawing it

Finally, let' solve the problem using VQE! We have written code in the block below to solve the problem. Don't worry about the code - focus on the results you get. 

The results will be displayed as follows:

`result:`

`optimal function value: 17.0`

`optimal value: [0. 1. 1. 1.]`

`status: SUCCESS`

`solution:`

 `[1, 2, 3]`
 
Here, `optimal function value: 17.0` is the optimized value of your knapsack found by VQE. `optimal value: [0. 1. 1. 1.]` shows which items you should put in your knapsack to get this value - 0 means that you should omit this item, and 1 means you should keep it. In this example, you should keep the items with index 1, 2, and 3, and omit the item with index 0 (remember indices begin at 0). This successful solution is also displayed separately as `solution: [1, 2, 3]` . 

Can you verify that VQE solves the knapsack problem correctly?

In [ ]:
# BLOCK 10 - Running VQE and printing results. 
calc = MinimumEigenOptimizer(method)
result = calc.solve(knapsack_problem.to_quadratic_program())
print('result:\n', result)
print('\nsolution:\n', knapsack_problem.interpret(result))

## Part 3 - Activities

1. Can you simplify the tunable circuit and still get the right result? Try reducing the number of repetitions and the level of entanglement in the tunable circuit. Do you still get the right answer?
2. Try changing the parameters (values, weights, max_weight). Does VQE still give you the optimal knapsack?

# Challenge activity

As we discussed in lecture, hybrid algorithms are an active area of research! VQE is not perfect, and can sometimes produce sub-optimal solutions. 

### **Can you find a combination of parameters (values, weights, max_weight) that breaks VQE?** 

**Hint:** Try making the problem larger! The bigger the problem, the harder it is to find an optimal solution